In [1]:
#!/usr/bin/env python3
# star_network_identify.py
# ---------------------------------------------------------------
"""
Identify star-like network topologies (A/B/C) from time-series produced by a
high-precision coupled map lattice (CML). The workflow is:

1) Simulate node dynamics on directed star graphs with high numerical precision.
2) Compute node-wise error strengths ‖x_{t+1} - 2 x_t‖ (wrapped on the circle).
3) Use a 2-component Gaussian Mixture Model (GMM) to separate hubs vs. leaves.
4) With two segments (from B/C candidates), compare hubs' residual variances to
   decide whether a segment came from type-B (both hubs fully connected) or
   type-C (leaves split across two hubs).

Conventions
-----------
- Adjacency A is binary and directed. A[j, i] = 1 means a directed edge j → i,
  i.e., node j contributes to the coupling term of node i.
- Trajectories are stored as a NumPy array of shape (N, T_seg), with N nodes
  and T_seg time steps after discarding transients.
"""

from __future__ import annotations
from decimal import Decimal, getcontext
import numpy as np
import mpmath as mp
from typing import Callable, Tuple
from sklearn.mixture import GaussianMixture








In [ ]:
#基础定义图动力系统

In [11]:
# ---------- Global High Precision Settings ----------
getcontext().prec = 200
mp.mp.dps = getcontext().prec
TWOPI = mp.mpf('6.283185307179586476925286766559')
SIGMA_H2 = 0.5  # ∫ h^2(0,y) dm(y)  (kept for reference; not used directly)
# -------------------- 小工具：安全转 Decimal --------------------
def _to_dec(x) -> Decimal:
    """用 str 路径避免二进制浮点误差。"""
    if isinstance(x, Decimal):
        return x
    return Decimal(str(x))
#定义图动力系统
# ------------------------------------------------------------------
#  I. High-Precision Network System (unchanged)
# ------------------------------------------------------------------
class GraphSystemDecimal:
    """
    Coupled map lattice on a directed graph with high-precision arithmetic.

    Each node evolves via a local map (default: doubling map x ↦ 2x mod 1),
    plus diffusive sinusoidal coupling from its in-neighbors.

    Parameters
    ----------
    A : np.ndarray
        Directed adjacency matrix of shape (N, N). A[j, i] = 1 indicates j → i.
    alpha : str, optional
        Coupling strength as a decimal string (for exact Decimal parsing).
    local_map : Callable[[Decimal], Decimal], optional
        Local map f(x). Defaults to doubling map `(2*x) % 1`.
    coupling_fn : Callable[[Decimal, Decimal], Decimal], optional
        Pairwise coupling c(x_s, x_t) from source s to target t. If None,
        uses a sinusoidal diffusive term `-sin(2π x_s) + sin(2π x_t)`.
    seed : int, optional
        Random seed for initial conditions.

    Attributes
    ----------
    N : int
        Number of nodes.
    Delta : float
        Maximum out-degree (max column sum) used for normalization.
    x : list[Decimal]
        Current node states.
    t : int
        Current time step.

    Notes
    -----
    - High precision is enforced via Python's Decimal and mpmath.
    - The coupling increment at node i is normalized by Delta.
    """

    def __init__(self, A: np.ndarray, alpha: str = '0.25',
                 local_map: Callable[[Decimal], Decimal] | None = None,
                 coupling_fn: Callable[[Decimal, Decimal], Decimal] | None = None,
                 seed: int = 0):
        self.A = np.asarray(A, dtype=float)
        self.N = self.A.shape[0]
        self.Delta = self.A.sum(axis=0).max()
        self.alpha = Decimal(alpha)
        self.local_map = local_map or (lambda x: (Decimal(2) * x) % 1)
        self.coupling = coupling_fn or self._default_coupling
        self.rng = np.random.default_rng(seed)
        self.reset()

    @staticmethod
    def _default_coupling(xs: Decimal, xt: Decimal) -> Decimal:
        """
        Default sinusoidal diffusive coupling.

        Parameters
        ----------
        xs : Decimal
            Source node state.
        xt : Decimal
            Target node state.

        Returns
        -------
        Decimal
            c(xs, xt) = -sin(2π xs) + sin(2π xt), as a Decimal.
        """
        v = -mp.sin(TWOPI * mp.mpf(str(xs))) + mp.sin(TWOPI * mp.mpf(str(xt)))
        return Decimal(str(v))

    def _coupling_term(self):
        """
        Compute normalized coupling increment for each node.

        Returns
        -------
        list[Decimal]
            A list of length N with the coupling increment for each node,
            normalized by the maximum out-degree Δ.

        Notes
        -----
        The increment for node i is the sum over j with A[j, i] = 1 of
        c(x_j, x_i), divided by Δ to keep scales comparable across graphs.
        """
        incr = [Decimal(0)] * self.N
        for j in range(self.N):
            if self.A[j].sum() == 0:
                continue  # node j has no outgoing edges
            for i in range(self.N):
                if self.A[j, i]:
                    incr[i] += self.coupling(self.x[j], self.x[i])
        d = Decimal(str(self.Delta))
        return [v / d for v in incr]

    def step(self):
        """
        Advance the system by one time step.

        Returns
        -------
        list[Decimal]
            The updated state vector x_{t+1} (length N) as Decimals.
        """
        xn = [self.local_map(x) for x in self.x]  # local map update
        coup = self._coupling_term()              # diffusive coupling
        xn = [(xi + self.alpha * ci) % 1 for xi, ci in zip(xn, coup)]
        self.x = xn
        self.t += 1
        return xn

    def reset(self):
        """
        Reset the system to a fresh random initial condition.

        Notes
        -----
        States are sampled i.i.d. ~ Uniform(0, 1) and stored as Decimal.
        """
        self.x = [Decimal(str(v)) for v in self.rng.random(self.N)]
        self.t = 0

    def run(self, T: int, discard: int = 0):
        """
        Simulate for T time steps and return the trajectory after discarding transients.

        Parameters
        ----------
        T : int
            Total number of steps to simulate.
        discard : int, optional
            Number of initial steps to discard as transients.

        Returns
        -------
        np.ndarray
            Array of shape (N, max(0, T - discard)) with float64 views of states.
        """
        traj = np.zeros((self.N, max(0, T - discard)))
        for k in range(T):
            xt = self.step()
            if k >= discard:
                traj[:, k - discard] = [float(v) for v in xt]
        return traj

In [ ]:
#图矩阵的生成，生成随机矩阵，包括严格ABC和ABC_like（B和C之间的图）和ABC族随机矩阵

In [3]:
# ---------------------------------------------------------------------
# Star graph generators (A: single hub; B: two hubs, all leaves to both;
# C: two hubs, leaves split into two disjoint halves)
# ---------------------------------------------------------------------
#生成区分的图矩阵
def graph_A(N: int) -> np.ndarray:
    """Star with one hub at index N-1 (all leaves point to the hub)."""
    A = np.zeros((N, N))
    A[np.arange(N - 1), N - 1] = 1
    return A


def graph_B(N: int) -> np.ndarray:
    """Two hubs at indices N-2 and N-1; every leaf connects to both hubs."""
    A = np.zeros((N, N))
    leaves = np.arange(N - 2)
    A[leaves, N - 1] = 1
    A[leaves, N - 2] = 1
    return A


def graph_C(N: int) -> np.ndarray:
    """
    Two hubs at indices N-2 and N-1; the leaves (0..N-3) are split evenly:
    first half -> hub N-2, second half -> hub N-1.
    """
    A = np.zeros((N, N))
    L = N - 2       # number of leaves
    half = L // 2
    first = np.arange(0, half)
    second = np.arange(half, L)
    A[first, N - 2] = 1
    A[second, N - 1] = 1
    return A

def graph_A_like(N: int) -> np.ndarray:
    """Star with one hub at index N-1 (all leaves point to the hub)."""
    A = np.zeros((N, N))
    A[np.arange(N - 1), N - 1] = 1
    return A


def graph_B_like(N: int) -> np.ndarray:
    """Two hubs at indices N-2 and N-1; every leaf connects to both hubs."""
    A = np.zeros((N, N))
    leaves = np.arange(N - 2)
    A[leaves, N - 1] = 1
    A[leaves, N - 2] = 1
    A_leaves=np.arange(0,int(N/3)-2)
    B_leaves=np.arange(int(N*2/3),N-2)
    A[A_leaves,N-1]=0
    A[B_leaves,N-2]=0
    return A


def graph_C_like(N: int) -> np.ndarray:
    """
    Two hubs at indices N-2 and N-1; the leaves (0..N-3) are split evenly:
    first half -> hub N-2, second half -> hub N-1.
    """
    A = np.zeros((N, N))
    L = N - 2       # number of leaves
    half = L // 2
    first = np.arange(0, half)
    second = np.arange(half, L)
    A[first, N - 2] = 1
    A[second, N - 1] = 1
    return A

In [ ]:
#GMM工具的函数，使用GMM模型对算法1的误差分类，以及对算法1.1的皮尔逊距离分类计算hub节点

In [12]:
# ------------------------------------------------------------------
#  III. GMM-based Hub Detection & Core Statistics
# ------------------------------------------------------------------
#GMM模型的定义
def moddiff(u):
    """
    Wrap a real array onto the interval (-0.5, 0.5] using modulo-1 arithmetic.

    Parameters
    ----------
    u : array_like
        Input values (can be scalar or array).

    Returns
    -------
    np.ndarray or float
        Wrapped values with the same shape as input.
    """
    return ((u + 0.5) % 1) - 0.5

# -------------------- 用 Decimal 包裹差值 d → (-0.5, 0.5] --------------------
def modwrap_diff_decimal_array(d: np.ndarray) -> np.ndarray:
    """
    输入 d 为 float64 数组（或可转 float 的 ndarray），逐点用 Decimal 计算：
        w = ((d + 0.5) % 1) - 0.5
    返回 float64（供后续 NumPy/Sklearn）。
    """
    d = np.asarray(d)
    out = np.empty_like(d, dtype=float)
    one = Decimal('1')
    half = Decimal('0.5')
    it = np.nditer(d, flags=['multi_index'])
    for val in it:
        dv = _to_dec(float(val))
        w  = (dv + half) % one - half
        out[it.multi_index] = float(w)
    return out


def gmm_hubs(S, seed=0):
    """
    Use a 2-component Gaussian Mixture Model to separate hubs (larger error
    strength) from leaves.

    Parameters
    ----------
    S : np.ndarray
        Mean error strengths for N nodes; shape (N,).
    seed : int, optional
        Random seed for GMM initialization.

    Returns
    -------
    np.ndarray
        Boolean mask of shape (N,), where True indicates a hub (the component
        with the larger mean).
    """
    g = GaussianMixture(2, random_state=seed).fit(S.reshape(-1, 1))
    return g.predict(S.reshape(-1, 1)) == np.argmax(g.means_)



#函数待修改,需要根据输入的参数H(x,y)和local_map(如果有local_map的遍历测度，那就用给定的不变测度)
#如果没有local_map或没有local_map的不变测度,就用给定的local_map(参数是local_map,可以是给定的，也可以是根据算法1.1拟合出来计算)
#用local_map的遍历测度去对H(x,y)进行积分估计
def beta_var(x: np.ndarray) -> float:
    """
    Estimate the variance of residuals in
    y_t = x_{t+1} - 2 x_t + β sin(2π x_t), via least squares for β.

    Parameters
    ----------
    x : np.ndarray
        Single-node time series of shape (T,).

    Returns
    -------
    float
        Variance of residuals y_t + β sin(2π x_t).
    """
    y = moddiff(x[1:] - 2 * x[:-1])
    s = -np.sin(2 * np.pi * x[:-1])
    beta = -(y @ s) / (s @ s)
    resid = y + beta * s
    return resid.var()

# ======== New or Replacement Section Begins =================================
# I. General Logistic Map (Decimal version, co-existing with original 2 x mod 1)
def logistic_map_decimal(x: Decimal) -> Decimal:
    """
    Logistic map in Decimal precision: f(x) = 4 x (1 - x)  (mod 1).

    Parameters
    ----------
    x : Decimal
        State value in [0, 1).

    Returns
    -------
    Decimal
        f(x) mapped back to [0, 1) using modulo-1, to match the original design.

    Notes
    -----
    Keeping the modulo-1 ensures consistency with the doubling-map implementation.
    """
    # “% 1” keeps the same format as in the original implementation
    return (Decimal(4) * x * (Decimal(1) - x)) % 1


# II. Example interface for an optional coupling function
def coupling_sin_diff(xs: Decimal, xt: Decimal) -> Decimal:
    """
    Sinusoidal diffusive coupling in Decimal precision:
    c(xs, xt) = -sin(2π xs) + sin(2π xt).

    Parameters
    ----------
    xs : Decimal
        Source node state.
    xt : Decimal
        Target node state.

    Returns
    -------
    Decimal
        Coupling contribution from xs to xt.
    """
    v = -mp.sin(TWOPI * mp.mpf(str(xs))) + mp.sin(TWOPI * mp.mpf(str(xt)))
    return Decimal(str(v))


# -------------------- 把任意 local_map 包装成可向量化版本 --------------------
# -------------------- 工厂：把“Decimal 标量 f”包装成“数组版 local_map_vec” --------------------
def make_local_map_vec_decimal(local_map_scalar: Callable[[Decimal], Decimal],
                               wrap_mod1: bool = True) -> Callable[[np.ndarray], np.ndarray]:
    """
    把标量版 local_map: Decimal -> Decimal，封装成数组版 local_map_vec(X)：
      - 逐点用 Decimal 计算；
      - 可选对输出再取 mod 1（适合 2x mod 1）；
      - 最终返回 float64 数组（后续 NumPy/Sklearn 友好）。

    参数
    ----
    local_map_scalar : Callable[[Decimal], Decimal]
        你的局部动力学（标量、Decimal）。
        例：doubling: lambda z: (Decimal(2)*z) % Decimal(1)
            logistic: lambda z:  Decimal(4)*z*(Decimal(1)-z)
    wrap_mod1 : bool
        若 local_map_scalar 没有自身取模，而你希望输出落在 [0,1)（如 2x mod 1），设 True。

    返回
    ----
    local_map_vec : Callable[[np.ndarray], np.ndarray]
        接受 float64 ndarray，返回 float64 ndarray。
    """
    one = Decimal('1')

    def local_map_vec(X: np.ndarray) -> np.ndarray:
        X = np.asarray(X)
        Y = np.empty_like(X, dtype=float)
        it = np.nditer(X, flags=['multi_index'])
        for x in it:
            xd = _to_dec(float(x))
            yd = local_map_scalar(xd)
            if wrap_mod1:
                yd = yd % one
            # 可选：保证在 [0,1] 内（避免极端误差）
            if yd < 0: yd = Decimal('0')
            if yd > 1: yd = Decimal('1')
            Y[it.multi_index] = float(yd)
        return Y

    return local_map_vec

# III. --- Modify compute_strength / beta_var so they depend on local_map ---
# -------------------- 强度计算（支持任意 local_map） --------------------
def compute_strength_with_local_map(
    traj: np.ndarray,
    local_map_vec: Callable[[np.ndarray], np.ndarray],
    wrap_mod1: bool = True,
) -> np.ndarray:
    """
    计算每个节点相对“已知局部映射 f”的平均绝对创新强度：
        S_i = ⟨ |Δ_i(t)| ⟩_t,
    其中 Δ_i(t) = x_{i,t+1} − f(x_{i,t})，再用 moddiff 包到 (-0.5, 0.5]。

    参数
    ----
    traj : np.ndarray
        形状 (N, T) 的轨迹矩阵（丢弃瞬态后）。
    local_map : Callable[[float], float]
        标量版的 f(x)（你也可以传入能处理 ndarray 的函数）。
    wrap_mod1 : bool
        若 f 的定义是“模 1”的（如 2x mod 1），置 True；否则 False。

    返回
    ----
    S : np.ndarray
        长度为 N 的强度向量。
    """
    x, x1 = traj[:, :-1], traj[:, 1:]
    # 原来：Delta = moddiff(x1 - local_map_vec(x))
    # 现在：用 Decimal 逐点包裹差值（先计算 float 差，再用 Decimal 做环上包裹）
    Delta = modwrap_diff_decimal_array(x1 - local_map_vec(x))
    return np.abs(Delta).mean(axis=1)


def beta_var(
    traj_i: np.ndarray,
    local_map_vec: Callable[[np.ndarray], np.ndarray],
    I_h_vec: Callable[[np.ndarray], np.ndarray]
) -> float:
    """
    Estimate β by least squares and return the residual variance for one node.

    Model
    -----
    y_t = x_{t+1} − f(x_t) + β · I_h(x_t),
    where I_h(x) = ∫ h(x, y) dm(y).
    All differences are wrapped via modulo-1 to stay on the circle.

    Parameters
    ----------
    traj_i : np.ndarray
        Single-node series of shape (T,).
    local_map_vec : Callable[[np.ndarray], np.ndarray]
        Vectorized local map f for arrays of shape (T-1,) → (T-1,).
    I_h_vec : Callable[[np.ndarray], np.ndarray]
        Vectorized function I_h for arrays of shape (T-1,) → (T-1,).

    Returns
    -------
    float
        Variance of residuals y + β · I_h(x).

    Notes
    -----
    β is obtained by minimizing ‖y + β s‖² with s = I_h(x), yielding
    β* = −(yᵀ s)/(sᵀ s).
    """
    x     = traj_i[:-1]
    y     = moddiff(traj_i[1:] - local_map_vec(x))
    s     = I_h_vec(x)
    beta  = -(y @ s) / (s @ s)
    resid = y + beta * s
    return resid.var()

In [13]:
# ------------------------------------------------------------------
#  IV-a  Single Segment → Classify A vs (B/C)
# ------------------------------------------------------------------
#算法1，区分A和BC图
def classify_A_and_BC_local_map_known(
    traj: np.ndarray,
    local_map_scalar: Callable[[Decimal], Decimal],
    wrap_mod1: bool = True,
    seed: int = 0,
    return_indices: bool = False,
) -> str | Tuple[str, np.ndarray]:
    """
    用“已知 f”的强度法区分 A 与 (B/C)：
      - 若恰有 1 个 hub → 'A_N'
      - 否则（通常是 2 个 hub） → 'B_N and C_N'

    参数
    ----
    traj : np.ndarray
        (N, T) 轨迹。
    local_map : Callable[[float], float]
        f(x)。例如：
            doubling:  lambda u: (2.0*u) % 1.0      （wrap_mod1=True）
            logistic:  lambda u: 4.0*u*(1.0-u)      （wrap_mod1=False）
    wrap_mod1 : bool
        f 是否需要取模 1（如 doubling 需要）。
    seed : int
        GMM 的随机种子。
    return_indices : bool
        若为 True，额外返回判为 hub 的节点索引。

    返回
    ----
    label : str
        'A_N' 或 'B_N and C_N'
    （可选）hubs : np.ndarray
        被判定为 hub 的节点索引（布尔簇的实际索引）。
    """
    local_map_vec = make_local_map_vec_decimal(local_map_scalar, wrap_mod1=wrap_mod1)
    S = compute_strength(traj, local_map_vec)
    hubs = np.where(gmm_hubs(S, seed=seed))[0]
    return "A_N" if hubs.size == 1 else "B_N and C_N"


#计算中心节点方差
def average_hub_variance(traj: np.ndarray) -> float:
    """
    Compute the mean residual variance across the two detected hubs
    for a B/C star graph, using the logistic local map and I_h.

    Parameters
    ----------
    traj : np.ndarray
        Trajectory array of shape (N, T).

    Returns
    -------
    float
        Average of beta_var over the two hubs.

    Raises
    ------
    RuntimeError
        If the number of detected hubs is not exactly two.
    """
    S    = compute_strength(traj, logistic_vec)
    hubs = np.where(gmm_hubs(S))[0]
    if hubs.size != 2:
        raise RuntimeError("Data are not from a B/C graph (number of hubs ≠ 2)")
    vars_ = [beta_var(traj[i], logistic_vec, Ih_vec) for i in hubs]
    return float(np.mean(vars_))

In [15]:
# VI. ------------------------ Demo -----------------------------------------

#算法1测试
if __name__ == "__main__":
    """
    Demo: run the system under the logistic map and sinusoidal coupling,
    then classify A/B/C for single segments and distinguish B vs C by variance.
    """
    N, T, discard = 10, 6000, 600
    alpha = '0.25'

    local_map = logistic_map_decimal
    local_map = lambda x:Decimal(2)*x%Decimal(1)
    coupling  = coupling_sin_diff

    # IV. --- Vectorized utilities, isolated from the Decimal system -------------
    # logistic_vec: elementwise version of f(x) = 4x(1-x) acting on ndarray inputs.
    #logistic_vec = np.vectorize(lambda u: 4.0 * u * (1.0 - u))          # f(x)

    # Ih_vec: elementwise version of I_h(x) = ∫ h(x, y) dm(y); here chosen as -sin(2πx).
    Ih_vec       = np.vectorize(lambda u: -np.sin(2 * np.pi * u))       # ∫ h dm
    # Demonstrate single-segment classification for A/B/C
    for gname, maker in [("A_N", graph_A),
                         ("B_N", graph_B),
                         ("C_N", graph_C)]:
        gs   = GraphSystemDecimal(maker(N), alpha=alpha,
                                  local_map=local_map,
                                  coupling_fn=coupling,
                                  seed=hash(gname) % 2**32)
        traj = gs.run(T, discard)
        print(f"{gname}  → classify_ABC → {classify_A_and_BC_local_map_known(traj,local_map_scalar=local_map)}")

    # Compare two sequences to distinguish between B and C
    trajB = GraphSystemDecimal(graph_B(N), alpha=alpha,
                               local_map=local_map,
                               coupling_fn=coupling,
                               seed=1).run(T, discard)
    trajC = GraphSystemDecimal(graph_C(N), alpha=alpha,
                               local_map=local_map,
                               coupling_fn=coupling,
                               seed=2).run(T, discard)
    
    
    
    
    
    
    

    #res, v_first, v_second = classify_B_vs_C(trajB, trajC)
    #print("\nCompare the two sequences:", res)
    #print(f"  Mean hub variance for the first sequence  = {v_first:.6e}")
    #print(f"  Mean hub variance for the second sequence = {v_second:.6e}")
# ======== New or Replacement Section Ends ===================================

A_N  → classify_ABC → A_N
B_N  → classify_ABC → B_N and C_N
C_N  → classify_ABC → B_N and C_N


In [ ]:
#算法1.2未知local_map的时候的映射

In [ ]:
# ----------------------------- Utilities ------------------------------------
def _angle(u: np.ndarray) -> np.ndarray:
    """Map x in [0,1) to 2πx angles."""
    return 2.0 * np.pi * u
def fourier_design(x: np.ndarray, M: int) -> np.ndarray:
    """
    Build a Fourier feature matrix Φ(x) = [cos(0*θ), cos(1*θ),...,cos(M*θ), sin(1*θ),...,sin(M*θ)],
    where θ = 2πx.

    Parameters
    ----------
    x : np.ndarray, shape (T,)
        Inputs in [0,1).
    M : int
        Number of harmonics.

    Returns
    -------
    Φ : np.ndarray, shape (T, 2M+1)
        Columns: [cos0,...,cosM, sin1,...,sinM].
        Note sin0 is identically 0 and omitted; cos0 = 1 is the intercept.
    """
    theta = _angle(x)
    T = x.shape[0]
    # cos block: j = 0..M
    cos_block = np.empty((T, M + 1))
    for j in range(M + 1):
        cos_block[:, j] = np.cos(j * theta)
    # sin block: j = 1..M
    sin_block = np.empty((T, M))
    for j in range(1, M + 1):
        sin_block[:, j - 1] = np.sin(j * theta)
    return np.concatenate([cos_block, sin_block], axis=1)  # (T, 2M+1)

@dataclass
class TrigMap:
    """
    A learned circular map represented by two linear models:
       sin(2π x_next) ≈ Φ(x) @ w_s
       cos(2π x_next) ≈ Φ(x) @ w_c
    """
    M: int
    w_s: np.ndarray  # shape (2M+1,)
    w_c: np.ndarray  # shape (2M+1,)

    def predict_sc(self, x: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Predict sin and cos of the next angle for a batch of inputs in [0,1).

        Returns
        -------
        (s_pred, c_pred) : two arrays of shape (len(x),)
        """
        Phi = fourier_design(x, self.M)
        s_pred = Phi @ self.w_s
        c_pred = Phi @ self.w_c
        return s_pred, c_pred

    def predict_next(self, x: np.ndarray) -> np.ndarray:
        """
        Predict next x in [0,1) by atan2 on predicted (sin, cos).

        Returns
        -------
        x_next_pred : np.ndarray in [0,1)
        """
        s_pred, c_pred = self.predict_sc(x)
        ang = np.arctan2(s_pred, c_pred)  # (-pi, pi]
        x_next = (ang / (2.0 * np.pi)) % 1.0
        return x_next

    
    

    
    
    
#逐个节点的函数用傅立叶拟合计算gi
def fit_trig_map_for_node(x: np.ndarray, y: np.ndarray, M: int, ridge: float = 0.0) -> TrigMap:
    """
    Fit a TrigMap for one node by linear least squares on Fourier features.

    Targets are circular: we regress sin(2π y) and cos(2π y) separately on Φ(x).

    Parameters
    ----------
    x : (T-1,)
        Inputs (current states).
    y : (T-1,)
        Outputs (next states).
    M : int
        Fourier order.
    ridge : float
        Optional L2-regularization strength (0.0 = ordinary least squares).

    Returns
    -------
    TrigMap
    """
    Phi = fourier_design(x, M)            # (T-1, 2M+1)
    s_tar = np.sin(_angle(y))             # (T-1,)
    c_tar = np.cos(_angle(y))             # (T-1,)

    if ridge > 0.0:
        # Ridge: (Phi^T Phi + λ I) w = Phi^T target
        G = Phi.T @ Phi + ridge * np.eye(Phi.shape[1])
        w_s = np.linalg.solve(G, Phi.T @ s_tar)
        w_c = np.linalg.solve(G, Phi.T @ c_tar)
    else:
        # Ordinary LS via lstsq (robust to mild collinearity)
        w_s, *_ = np.linalg.lstsq(Phi, s_tar, rcond=None)
        w_c, *_ = np.linalg.lstsq(Phi, c_tar, rcond=None)

    return TrigMap(M=M, w_s=w_s, w_c=w_c)


def fit_trig_map_per_node(traj: np.ndarray, M: int, ridge: float = 0.0) -> list[TrigMap]:
    """
    Fit a TrigMap g_i for each node i from its trajectory x_i(t) -> x_i(t+1).

    Parameters
    ----------
    traj : np.ndarray, shape (N, T)
    M : int
        Fourier order.
    ridge : float
        Optional ridge regularization.

    Returns
    -------
    models : list[TrigMap] of length N
    """
    N, T = traj.shape
    models: list[TrigMap] = []
    for i in range(N):
        x = traj[i, :-1]
        y = traj[i,  1:]
        models.append(fit_trig_map_for_node(x, y, M=M, ridge=ridge))
    return models

In [ ]:
#映射到圆上降低数值误差的问题
def evaluate_models_on_grid(models: list[TrigMap], G: int = 2048) -> np.ndarray:
    """
    Evaluate each model (sin and cos) on a uniform grid of size G in [0,1).

    Returns
    -------
    V : np.ndarray, shape (N, 2G)
        For node i, row i is [s_pred(grid), c_pred(grid)] concatenated.
        We also L2-normalize per grid point to reduce amplitude effects.
    """
    N = len(models)
    grid = np.linspace(0.0, 1.0, G, endpoint=False)
    V = np.zeros((N, 2 * G), dtype=float)
    for i, m in enumerate(models):
        s, c = m.predict_sc(grid)  # each shape (G,)
        # Normalize (optional but helpful): project to unit circle per grid point
        norm = np.sqrt(s * s + c * c) + 1e-12
        s_n = s / norm
        c_n = c / norm
        V[i, :G] = s_n
        V[i, G:] = c_n
    return V


def pearson_distance_matrix(V: np.ndarray) -> np.ndarray:
    """
    Compute pairwise Pearson distances on rows of V:
        d(i,j) = 1 - corr(V_i, V_j).

    Parameters
    ----------
    V : np.ndarray, shape (N, D)

    Returns
    -------
    Dmat : np.ndarray, shape (N, N)
        Symmetric with zeros on the diagonal.
    """
    # Center columns
    X = V - V.mean(axis=0, keepdims=True)      # (N, D)
    # Row-wise norms
    row_norm = np.linalg.norm(X, axis=1, keepdims=True) + 1e-12
    Xn = X / row_norm
    # Correlation matrix = Xn Xn^T
    C = Xn @ Xn.T
    # Distance = 1 - corr
    Dmat = 1.0 - C
    np.fill_diagonal(Dmat, 0.0)
    return Dmat


def hub_scores_from_distances(Dmat: np.ndarray) -> np.ndarray:
    """
    Hub score S_i = sum_{j≠i} d(i,j): how dissimilar node i is from the rest.

    Parameters
    ----------
    Dmat : np.ndarray, (N, N)

    Returns
    -------
    S : np.ndarray, (N,)
    """
    return Dmat.sum(axis=1)

In [ ]:
#区分A和BC类在未知f的情况下的代码

In [16]:
def detect_hubs_from_scores(S: np.ndarray, seed: int = 0) -> np.ndarray:
    """
    Detect hubs by a 2-component GMM on the scores S.
    The component with the larger mean corresponds to "hubs".

    Returns
    -------
    hubs_mask : np.ndarray of bool, shape (N,)
    """
    gmm = GaussianMixture(2, random_state=seed).fit(S.reshape(-1, 1))
    labels = gmm.predict(S.reshape(-1, 1))
    means = gmm.means_.flatten()
    hub_label = np.argmax(means)
    hubs_mask = (labels == hub_label)
    return hubs_mask


# ----------------------- Main: A vs (B/C) when f is unknown -----------------
# ------------------------ 主函数：未知 f 的 A vs (B/C) ------------------------
def classify_A_vs_BC_unknown_f(
    traj: np.ndarray,
    M: int = 10,
    grid_size: int = 2048,
    ridge: float = 0.0,
    seed: int = 0,
    return_indices: bool = True,
    return_fhat: bool = True,
) -> str | Tuple[str, np.ndarray] | Dict[str, Any]:
    """
    用“未知 f”的函数不相似度法区分 A 与 (B/C)；可选返回 hub 索引与叶子拟合的 f̂。

    参数
    ----
    traj : np.ndarray (N x T)
        节点轨迹（已去瞬态）。
    M : int
        傅里叶阶数（推荐 8~12 做验证选型）。
    grid_size : int
        比较函数用的网格大小（默认 2048）。
    ridge : float
        Ridge 正则强度（0 表示普通最小二乘）。
    seed : int
        GMM 随机种子。
    return_indices : bool
        若 True，返回 (label, hubs_idx)。
    return_fhat : bool
        若 True，返回包含 f_hat（叶子拟合）的详细字典。

    返回
    ----
    - 若两者均 False：返回 str：'A_N' 或 'B_N and C_N'
    - 若 return_indices=True 且 return_fhat=False：
         返回 (label:str, hubs_idx:np.ndarray)
    - 若 return_fhat=True（无论 return_indices True/False）：
         返回 dict，包含：
            'label'           : 'A_N' / 'B_N and C_N'
            'hubs_idx'        : hub 索引
            'leaves_idx'      : 叶子索引
            'scores'          : S_i
            'distance_matrix' : Pearson 距离矩阵
            'models'          : 每个节点的 TrigMap
            'f_hat'           : 用叶子样本池化拟合得到的 TrigMap
    """
    N, T = traj.shape

    # (1) 逐节点拟合经验映射 g_i
    models = fit_trig_map_per_node(traj, M=M, ridge=ridge)

    # (2) 在统一网格上评估 (3) Pearson 距离矩阵 (4) 分数 S
    V = evaluate_models_on_grid(models, G=grid_size)
    Dmat = pearson_distance_matrix(V)
    S = hub_scores_from_distances(Dmat)

    # (4) 用你给的 GMM 选择 hub
    hubs_mask = detect_hubs_from_scores(S, seed=seed)
    hubs_idx = np.where(hubs_mask)[0]
    leaves_idx = np.where(~hubs_mask)[0]

    # (5) A vs (B/C)
    label = "A_N" if hubs_idx.size == 1 else "B_N and C_N"

    # ---- 返回逻辑（完全按你的开关）----
    if not return_indices and not return_fhat:
        return label

    if return_indices and not return_fhat:
        return label, hubs_idx

    # return_fhat=True（无论 return_indices True/False），返回详细字典
    if leaves_idx.size == 0:
        # 极少数情况下没有“叶子”，退化为非 hub（理论上不该发生，但加一道保险）
        leaves_idx = np.setdiff1d(np.arange(N), hubs_idx)

    # (6) 叶子样本池化拟合 f̂
    x_pool = traj[leaves_idx, :-1].ravel()
    y_pool = traj[leaves_idx,  1:].ravel()
    f_hat = fit_trig_map_for_node(x_pool, y_pool, M=M, ridge=ridge)

    return {
        "label": label,
        "hubs_idx": hubs_idx,
        "leaves_idx": leaves_idx,
        "scores": S,
        "distance_matrix": Dmat,
        "models": models,
        "f_hat": f_hat,
    }

In [17]:
# ----------------------- Demo (if you want to test) -------------------------
if __name__ == "__main__":
    # Example demo assuming you have GraphSystemDecimal, graph_A/B/C available
    # and coupling_sin_sin (h = 2 sin x sin y). Here we just show usage.
    from math import isfinite

    N, T, discard = 40, 6000, 600
    alpha = "0.25"

    # Simulate one A, one B, one C segment (with known simulator but unknown f to the classifier)
    gsA = GraphSystemDecimal(graph_A(N), alpha=alpha, seed=1)
    trajA = gsA.run(T, discard)

    gsB = GraphSystemDecimal(graph_B(N), alpha=alpha, seed=2)
    trajB = gsB.run(T, discard)

    gsC = GraphSystemDecimal(graph_C(N), alpha=alpha, seed=3)
    trajC = gsC.run(T, discard)

    # Classify A vs (B/C) with unknown f
    outA = classify_A_vs_BC_unknown_f(trajA, M=10, grid_size=2048, seed=0)
    print("Segment A classified as:", outA["label"], "| hubs:", outA["hubs_idx"])

    outB = classify_A_vs_BC_unknown_f(trajB, M=10, grid_size=2048, seed=0)
    print("Segment B classified as:", outB["label"], "| hubs:", outB["hubs_idx"])

    outC = classify_A_vs_BC_unknown_f(trajC, M=10, grid_size=2048, seed=0)
    print("Segment C classified as:", outC["label"], "| hubs:", outC["hubs_idx"])

    # Evaluate reconstructed f̂ from leaves on a grid (optional sanity check)
    grid = np.linspace(0, 1, 16, endpoint=False)
    fA_grid = evaluate_f_hat(outA["f_hat"], grid)
    print("A: f-hat(grid) head:", fA_grid[:5])

NameError: name 'fit_trig_map_per_node' is not defined